In [1]:
!pip install pandas langchain langchain-community sentence-transformers faiss-cpu openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.

In [2]:
import pandas as pd
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
import os

# --- Константы ---
CSV_FILE_PATH = "historical_events.csv" # Убедитесь, что имя файла совпадает с загруженным
EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
INDEX_SAVE_PATH = "faiss_index_historical" # Папка, куда будет сохранен индекс

In [3]:
def load_documents_from_csv(file_path):
    """Загружает данные из CSV и преобразует в документы LangChain."""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Файл данных не найден: {file_path}")
    try:
        df = pd.read_csv(file_path)
        documents = []
        for index, row in df.iterrows():
            content = f"Дата: {row['date']}. Событие: {row['event_description']}"
            if pd.notna(row.get('location')):
                 content += f". Место: {row['location']}"
            if pd.notna(row.get('category')):
                 content += f". Категория: {row['category']}"

            metadata = {"date": str(row['date']), "source": file_path}
            if pd.notna(row.get('location')):
                metadata["location"] = row['location']
            if pd.notna(row.get('category')):
                metadata["category"] = row['category']

            documents.append(Document(page_content=content, metadata=metadata))
        print(f"Загружено {len(documents)} документов из {file_path}")
        return documents
    except Exception as e:
        print(f"Ошибка при чтении или обработке CSV файла {file_path}: {e}")
        raise

# Загружаем документы
try:
    print("Загрузка документов из CSV...")
    documents = load_documents_from_csv(CSV_FILE_PATH)
except FileNotFoundError as fnf:
    print(f"Ошибка: {fnf}. Убедитесь, что файл CSV загружен в Colab.")
    # Остановить выполнение, если файла нет
    raise SystemExit("Остановка: CSV файл не найден.")
except Exception as e:
    print(f"Непредвиденная ошибка при загрузке документов: {e}")
    raise SystemExit("Остановка: Ошибка загрузки документов.")

if not documents:
    raise SystemExit("Остановка: Документы не были загружены или файл CSV пуст.")

Загрузка документов из CSV...
Загружено 80 документов из historical_events.csv


In [4]:
print(f"Инициализация модели эмбеддингов: {EMBEDDING_MODEL_NAME}...")
try:
    embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)
    print("Модель эмбеддингов загружена.")
except Exception as e:
    print(f"Ошибка загрузки модели эмбеддингов: {e}")
    raise SystemExit("Остановка: Ошибка модели эмбеддингов.")


print("Создание эмбеддингов для документов и индекса FAISS...")
# Это может занять некоторое время в зависимости от размера CSV
try:
    vector_store = FAISS.from_documents(documents, embeddings)
    print("Индекс FAISS успешно создан в памяти.")
except Exception as e:
    print(f"Ошибка при создании индекса FAISS: {e}")
    raise SystemExit("Остановка: Ошибка создания индекса.")

Инициализация модели эмбеддингов: sentence-transformers/paraphrase-multilingual-mpnet-base-v2...


<ipython-input-4-1958a17a6692>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Модель эмбеддингов загружена.
Создание эмбеддингов для документов и индекса FAISS...
Индекс FAISS успешно создан в памяти.


In [5]:
print(f"Сохранение индекса FAISS в папку: {INDEX_SAVE_PATH}...")
try:
    vector_store.save_local(folder_path=INDEX_SAVE_PATH)
    print(f"Индекс успешно сохранен. В папке '{INDEX_SAVE_PATH}' должны быть файлы 'index.faiss' и 'index.pkl'.")
except Exception as e:
    print(f"Ошибка при сохранении индекса FAISS: {e}")
    raise SystemExit("Остановка: Ошибка сохранения индекса.")

Сохранение индекса FAISS в папку: faiss_index_historical...
Индекс успешно сохранен. В папке 'faiss_index_historical' должны быть файлы 'index.faiss' и 'index.pkl'.
